In [1]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt

MPX_SHAPE = (5,5) # 3D-One Avior macropixel shape
MPX_CORNER = [1,1] # macropixel corner location for a 250x250 image (see macropxCorner MATLAB function), in 0-base indexing!
WL_PEAK = np.array([918, 926, 936, 945, 954, 858, 870, 884, 895, 904, 794, 810, 820, 833, 848, 730, 742, 755, 771, 784, 731, 742, 690, 703, 714])

def mosaic_mask(wl_idx, img):
    """
    Extract one debayered channel/wavelength from a hyperspectral mosaic image:
    1. Clips incomplete macropixels: 250x250 --> 245x245
    2. Extracts one pixel per macropixel to reconstruct one 49x49 image for the selected wavelength: 245x245 --> 49x49x25
    """
    img = img[MPX_CORNER[0]:img.shape[0], MPX_CORNER[1]:img.shape[1]] # first the image is cropped from the corner of the 1st macropixel
    img = img[0:img.shape[0]-np.mod(img.shape[0],MPX_SHAPE[0]), 0:img.shape[1]-np.mod(img.shape[1],MPX_SHAPE[1])] # then the image is cropped after the last full macropixel
    masked_im_shape = [int(img.shape[0]/MPX_SHAPE[0]), int(img.shape[1]/MPX_SHAPE[1])] # image length and width are divided by the macropixel length and width
    mpx_mask = np.zeros(MPX_SHAPE)
    mpx_mask[np.unravel_index([wl_idx], MPX_SHAPE)]=1
    # wavelength ordering (see 3D-One Avior manual and mosaic code documentation):
    # >>> for i in range(25):
    # ...     filter[np.unravel_index([i],[5,5])]=i;
    # >>> filter
    # array([[ 0.,  1.,  2.,  3.,  4.],
    #        [ 5.,  6.,  7.,  8.,  9.],
    #        [10., 11., 12., 13., 14.],
    #        [15., 16., 17., 18., 19.],
    #        [20., 21., 22., 23., 24.]])

    img_mask = np.tile(mpx_mask, masked_im_shape).astype(bool) # repeat the macropixel mask to cover whole image
    masked_img = img[img_mask].reshape(masked_im_shape)
    return masked_img

def debayer(img):
    """ Debayer an [H,W] image to a [H,W,C] shape (ToTensor() input shape)
    """
    deb_img0 = mosaic_mask(0, img) # get 1st image to know size
    deb_img = np.zeros((deb_img0.shape[0], deb_img0.shape[1], 25), dtype=np.uint8) # preallocate debayered image
    deb_img[:,:,0] = deb_img0
    for i in range(1,25):
        deb_img[:,:,i] = mosaic_mask(i,img) # debayer each wavelength
    return np.mean(deb_img, axis=2).astype(np.uint8)